# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 75 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.21551
extracting tarball to tmp_2507.21551... done.
Retrieving document from  https://arxiv.org/e-print/2507.21632


extracting tarball to tmp_2507.21632... done.


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure NBursts_sdss_101229_60056_27021598151649911_emiles_n1b1.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1.\linewidth]{NBursts_sdss_101229_60056_27021598151649911_emiles_n1b1.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure NBursts_sdss_103950_59937_27021598052190132_emiles_n1b1.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1.\linew

Found 86 bibliographic references in tmp_2507.21632/aa55117-25.bbl.
Issues with the citations
syntax error in line 107: '}' expected
Retrieving document from  https://arxiv.org/e-print/2507.21866


extracting tarball to tmp_2507.21866... done.
Retrieving document from  https://arxiv.org/e-print/2507.22006
extracting tarball to tmp_2507.22006... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.21632-b31b1b.svg)](https://arxiv.org/abs/2507.21632) | **Optical Emission-Line Properties of eROSITA-selected SDSS-V Galaxies**  |
|| N. G. Pulatova, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>M. Demianenko</mark> |
|*Appeared on*| *2025-07-30*|
|*Comments*| *15 pages, 9 figures, 3 tables*|
|**Abstract**|            We present and discuss optical emission line properties obtained from the analysis of Sloan Digital Sky Survey (SDSS) spectra for an X-ray selected sample of 3684 galaxies (0.002 < z < 0.55), drawn from the eRASS1 catalog. We modeled SDSS-V DR19 spectra using the NBursts full spectrum fitting technique with E-MILES simple stellar populations (SSP) models and emission line templates to decompose broad and narrow emission line components for correlation with X-ray properties. We place the galaxies on the Baldwin-Phillips-Terlevich (BPT) diagram to diagnose their dominant excitation mechanism. We show that the consistent use of the narrow component fluxes shifts most galaxies systematically and significantly upward to the active galactic nuclei (AGN) region on the BPT diagram. On this basis, we confirm the dependence between a galaxys position on the BPT diagram and its (0.2-2.3 keV) X-ray/H$\alpha$ flux ratio. We also verified the correlation between X-ray luminosity and emission line luminosities of the narrow [O\iii]$\lambda 5007$ and broad H$\alpha$ component; as well as the relations between the Supermassive Black Hole (SMBH) mass, the X-ray luminosity, and the velocity dispersion of the stellar component ($\sigma_{*}$) on the base on the unique sample of optical spectroscopic follow-up of X-ray sources detected by eROSITA. These results highlight the importance of emission line decomposition in AGN classification and refine the connection between X-ray emission and optical emission line properties in galaxies.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.21551-b31b1b.svg)](https://arxiv.org/abs/2507.21551) | **A North-South Metallicity Asymmetry in the Outer Galactic disk -- Evidence for the Pericentric Passage of the Sagittarius Dwarf Galaxy**  |
|| C. Wang, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-07-30*|
|*Comments*| *12pages, 8figures, accepted by ApJL*|
|**Abstract**|            We present maps of the mean metallicity distributions on the Galactocentric $R$--$Z$ plane at different azimuthal angles using red clump stars selected from the LAMOST and APOGEE surveys. In the inner disk ($R < $ 11\,kpc), the metallicity distribution is symmetric between the upper and lower disk. However, we find a North-South metallicity asymmetry in the outer disk ($R > 11$\,kpc), especially towards the anti-Galactic center ($-5^\circ < \Phi < 15^\circ$) direction. By further dissecting the map in age space, we detect this asymmetry across all mono-age stellar populations. However, the asymmetry is less pronounced in older populations ($\tau > 8$ Gyr) compared to younger ones ($\tau < 6$\,Gyr). This reduced significance likely stems from three factors: larger age uncertainties, fewer stars in the outer disk, and the kinematically hotter nature of older populations. The observed metallicity asymmetry may be the consequence of the purturbation of the recent pericentric passage through the Galactic disk and tidal force of the well-known Sagittarius dwarf galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.21866-b31b1b.svg)](https://arxiv.org/abs/2507.21866) | **SARD: A YOLOv8-Based System for Solar Active Region Detection with SDO/HMI Magnetograms**  |
|| J. Pan, <mark>J. Liu</mark>, S. Fang, R. Liu |
|*Appeared on*| *2025-07-30*|
|*Comments*| *19 pages*|
|**Abstract**|            Solar active regions are where sunspots are located and photospheric magnetic fluxes are concentrated, therefore being the sources of energetic eruptions in the solar atmosphere. The detection and statistics of solar active regions have been forefront topics in solar physics. In this study, we developed a solar active region detector (SARD) based on the advanced object detection model YOLOv8. First, we applied image processing techniques including thresholding and morphological operations to 6975 line-of-sight magnetograms from 2010 to 2019 at a cadence of 12~h, obtained by the Helioseismic and Magnetic Imager onboard the Solar Dynamic Observatory. With manual refinement, we labeled 26531 active regions in the dataset for further training and test with the detection model. Without any overlap between the training and test sets, the superior performance of SARD is demonstrated by an average precision rate as high as 94\%. We then performed a statistical analysis on the area and magnetic flux of the detected active regions, both of which yield log-normal distributions. This result sheds light on the underlying complexity and multi-scale nature of solar active regions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.22006-b31b1b.svg)](https://arxiv.org/abs/2507.22006) | **Probing ALP-Photon Mixing with High-Resolution X-ray Spectroscopy**  |
|| Y. Zhou, <mark>J. Liu</mark>, V. Takhistov, K. Mitsuda |
|*Appeared on*| *2025-07-30*|
|*Comments*| *16 pages, 10 figures*|
|**Abstract**|            Axion-like particles (ALPs) provide a compelling avenue for exploring physics beyond the Standard Model. In astrophysical magnetized plasmas an ALP-photon coupling $g_{a\gamma}$ induces energy-dependent oscillations in the photon survival probability that imprint modulations on emission spectra. X-ray observations of bright spectrally-smooth sources can provide particularly sensitive probes of ultralight ALPs with masses $m_a \lesssim 10^{-11}$ eV due to long propagation distances, strong magnetic fields and high photon statistics. We present a comprehensive forecast of ALP-photon conversion in three representative systems: (i) background active galactic nuclei (AGNs) observed through foreground intracluster magnetic fields, (ii) central AGNs within their host cluster halos and (iii) Galactic X-ray binaries viewed through the Milky Way field. Using detailed simulations we assess the prospective sensitivity of high-resolution X-ray missions including XRISM, Athena, and Arcus. For typical magnetic field configurations a 5 Ms XRISM observation of the Perseus Cluster AGN NGC 1275 can reach down to $g_{a\gamma} \sim 3 \times 10^{-13}$ GeV$^{-1}$ at $m_a \lesssim 10^{-12}$ eV, while Athena's superior energy resolution improves this reach by a factor of $\sim 3$. We quantify the impact of magnetic field modeling, photon statistics, and spectral binning strategies. Our results demonstrate the scientific potential of high-resolution X-ray observations to probe photon-ALP coupling in previously inaccessible parameter space, offering a powerful window into physics beyond the Standard Model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.21632/./L_H-alpha-NBursts-br-to-nar.jpg', 'tmp_2507.21632/./L_H-beta-NBursts-br-to-nar.jpg', 'tmp_2507.21632/./SPIDERS.png', 'tmp_2507.21632/./L-X-ray-M-BH-SDSS-V-col-pl.png']
copying  tmp_2507.21632/./L_H-alpha-NBursts-br-to-nar.jpg to _build/html/
copying  tmp_2507.21632/./L_H-beta-NBursts-br-to-nar.jpg to _build/html/
copying  tmp_2507.21632/./SPIDERS.png to _build/html/
copying  tmp_2507.21632/./L-X-ray-M-BH-SDSS-V-col-pl.png to _build/html/
exported in  _build/html/2507.21632.md
    + _build/html/tmp_2507.21632/./L_H-alpha-NBursts-br-to-nar.jpg
    + _build/html/tmp_2507.21632/./L_H-beta-NBursts-br-to-nar.jpg
    + _build/html/tmp_2507.21632/./SPIDERS.png
    + _build/html/tmp_2507.21632/./L-X-ray-M-BH-SDSS-V-col-pl.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ii}{ {\sc ii}}$
$\newcommand{\iii}{ {\sc iii}}$</div>



<div id="title">

# Optical Emission-Line Properties of \ eROSITA-selected SDSS-V Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.21632-b31b1b.svg)](https://arxiv.org/abs/2507.21632)<mark>Appeared on: 2025-07-30</mark> -  _15 pages, 9 figures, 3 tables_

</div>
<div id="authors">

N. G. Pulatova, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>M. Demianenko</mark>

</div>
<div id="abstract">

**Abstract:** We present and discuss optical emission line properties obtained from the analysis of Sloan Digital Sky Survey (SDSS) spectra for an X-ray selected sample of 3684 galaxies ( $0.002 < z < 0.55$ ), drawn from the eRASS1 catalog.We modeled SDSS-V DR19 spectra using the ${\sc NBursts}$ full spectrum fitting technique with E-MILES simple stellar populations (SSP) models and emission line templates to decompose broad and narrow emission line components for correlation with X-ray properties.We place the galaxies on the Baldwin-Phillips-Terlevich (BPT) diagram to diagnose their dominant excitation mechanism.We show that the consistent use of the narrow component fluxesshifts most galaxies systematically and significantly upward to the active galactic nuclei (AGN) region on the BPT diagram.On this basis, we confirm the dependence between a galaxy’s position on the BPT diagram and its ( $0.2-2.3$ keV) X-ray/H $\alpha$ flux ratio.We also verified the correlation between X-ray luminosity and emission line luminosities of the narrow [ O $\iii$ ] $\lambda 5007$ and broad H $\alpha$ component; as well as the relations between the Supermassive Black Hole (SMBH) mass, the X-ray luminosity, and the velocity dispersion of the stellar component ( $\sigma_{*}$ ) on the base on the unique sample of optical spectroscopic follow-up of X-ray sources detected by eROSITA.These results highlight the importance of emission line decomposition in AGN classification and refine the connection between X-ray emission and optical emission line properties in galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2507.21632/./L_H-alpha-NBursts-br-to-nar.jpg" alt="Fig8.1" width="50%"/><img src="tmp_2507.21632/./L_H-beta-NBursts-br-to-nar.jpg" alt="Fig8.2" width="50%"/>

**Figure 8. -** 
        Optical luminosity in the broad and narrow components. Color code: eROSITA X-ray luminosity in $0.2 - 2.3$ keV. The black dashed line represents 1:1 relation. It demonstrates a strong connection between the BLR, NLR, and the X-ray source. Panel a: H$\alpha$; Panel b: H$\beta$.
         (*Fig-Lum-nar-broad_X*)

</div>
<div id="div_fig2">

<img src="tmp_2507.21632/./SPIDERS.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
        Step-by-step description of samples selection.
         (*Fig-sam-sel*)

</div>
<div id="div_fig3">

<img src="tmp_2507.21632/./L-X-ray-M-BH-SDSS-V-col-pl.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
        The X-ray luminosity --- SMBH mass diagram. Color code: velocity dispersion of the stellar component $\sigma_{*}$.
         (*Fig_M_BH_LX*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.21632"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
